In [1]:
import shutil
import random
import os
import numpy as np
from PIL import Image

In [2]:
output = "./data"
sets = ["train", "valid", "test"]

folder_input = "./COT_AD1"
classes = os.listdir(folder_input)
print(classes)

['Fresh_Leaf', 'Leaf_Reddening', 'Leaf_Spot_Bacterial_Blight', 'Yellowish_Leaf']


In [8]:

# filtrar las imagenes tal que sean resolucion mayores a 255x255
folder_filtered = "./COT_AD1_filtered"
for cls in classes:
    os.makedirs(os.path.join(folder_filtered, cls), exist_ok=True)

# Filtrar imágenes por resolución
for cls in classes:
    folder_cls = os.path.join(folder_input, cls)
    filtered_images = []
    for img in os.listdir(folder_cls):
        img_path = os.path.join(folder_cls, img)
        with Image.open(img_path) as im:
            if im.size[0] > 100 and im.size[1] > 100:
                filtered_images.append(img)
    # Copiar imágenes filtradas a la nueva carpeta
    for img in filtered_images:
        src = os.path.join(folder_cls, img)
        dst = os.path.join(folder_filtered, cls, img)
        shutil.copy2(src, dst)

In [ ]:
import os
import random
from PIL import Image, ImageEnhance, ImageOps
import numpy as np
from tqdm import tqdm # Para una bonita barra de progreso

# ==============================================================================
# FUNCIONES DE AUMENTACIÓN DE IMÁGENES (SIN TORCH)
# ==============================================================================

def random_flip(image):
    """Voltea la imagen horizontalmente con un 50% de probabilidad."""
    if random.random() > 0.5:
        return ImageOps.mirror(image)
    #vertical_flip
    if random.random() > 0.5:
        return ImageOps.flip(image)
    return image

def random_rotation(image, max_angle=20):
    """Rota la imagen un ángulo aleatorio entre -max_angle y +max_angle."""
    angle = random.uniform(-max_angle, max_angle)
    # 'expand=True' asegura que la imagen no se recorte tras la rotación.
    # El color de relleno (fillcolor) puede ser negro o un gris neutro.
    return image.rotate(angle, resample=Image.Resampling.BICUBIC, expand=True)

def random_color_jitter(image):
    """Ajusta aleatoriamente el brillo, contraste y saturación."""
    # Brillo
    enhancer = ImageEnhance.Brightness(image)
    image = enhancer.enhance(random.uniform(0.7, 1.3))
    # Contraste
    enhancer = ImageEnhance.Contrast(image)
    image = enhancer.enhance(random.uniform(0.7, 1.3))
    # Saturación
    enhancer = ImageEnhance.Color(image)
    image = enhancer.enhance(random.uniform(0.7, 1.3))
    return image

def random_zoom(image, min_zoom=0.8, max_zoom=1.2):
    """Aplica un zoom aleatorio a la imagen."""
    width, height = image.size
    zoom_factor = random.uniform(min_zoom, max_zoom)
    
    new_width, new_height = int(width * zoom_factor), int(height * zoom_factor)
    
    # Redimensionar la imagen
    zoomed_image = image.resize((new_width, new_height), Image.Resampling.BICUBIC)
    
    # Recortar al tamaño original desde el centro
    left = (new_width - width) / 2
    top = (new_height - height) / 2
    right = (new_width + width) / 2
    bottom = (new_height + height) / 2
    
    return zoomed_image.crop((left, top, right, bottom))

In [16]:
'''
CLASS 1 -> total: 786 (train: 573, valid: 70, test: 143)
CLASS 2 -> total: 225 (train: 124, valid: 70, test: 31)
CLASS 3 -> total: 1303 (train: 986, valid: 70, test: 247)
CLASS 4 -> total: 135 (train: 52, valid: 70, test: 13)
'''


# Datos de distribución por clase
num_images = {
    "Fresh_Leaf": {"train": 708, "valid": 30, "test": 30},
    "Leaf_Reddening": {"train": 165, "valid": 30, "test": 30},
    "Leaf_Spot_Bacterial_Blight": {"train": 1243, "valid": 30, "test": 30},
    "Yellowish_Leaf": {"train": 75, "valid": 30, "test": 30},
}

# Crear directorios y copiar imágenes
for cls in classes:
    # Crear directorios
    for s in sets:
        os.makedirs(os.path.join(output, s, cls), exist_ok=True)
    
    # Obtener y mezclar imágenes
    images = os.listdir(os.path.join(folder_input, cls))
    random.shuffle(images)
    
    # Calcular índices
    train_count = num_images[cls]["train"]
    valid_count = num_images[cls]["valid"]
    test_count = num_images[cls]["test"]
    
    # Dividir las imágenes en conjuntos
    train_images = images[:train_count]
    valid_images = images[train_count:train_count + valid_count]
    test_images = images[train_count + valid_count:train_count + valid_count + test_count]

    os.makedirs(os.path.join(output, "train", cls), exist_ok=True)
    os.makedirs(os.path.join(output, "valid", cls), exist_ok=True)
    os.makedirs(os.path.join(output, "test", cls), exist_ok=True)
    
    # Copiar imágenes de train
    for img in train_images:
        src = os.path.join(folder_input, cls, img)
        dst = os.path.join(output, "train", cls, img)
        shutil.copy2(src, dst)
    
    # Copiar imágenes de valid
    for img in valid_images:
        src = os.path.join(folder_input, cls, img)
        dst = os.path.join(output, "valid", cls, img)
        shutil.copy2(src, dst)
    
    # Copiar imágenes de test
    for img in test_images:
        src = os.path.join(folder_input, cls, img)
        dst = os.path.join(output, "test", cls, img)
        shutil.copy2(src, dst)
    
    print(f"Completed copying for class: {cls}")

Completed copying for class: Fresh_Leaf
Completed copying for class: Leaf_Reddening
Completed copying for class: Leaf_Spot_Bacterial_Blight
Completed copying for class: Yellowish_Leaf


In [17]:
# --- CONFIGURACIÓN ---
# Ruta a tu dataset filtrado pero aún no balanceado
input_folder = "./data/train"
# La clase con más imágenes (1290) define nuestro objetivo
target_count = 1290

# Obtenemos las clases y sus conteos actuales
classes = [d for d in os.listdir(input_folder) if os.path.isdir(os.path.join(input_folder, d))]
class_counts = {cls: len(os.listdir(os.path.join(input_folder, cls))) for cls in classes}

print("--- Conteo de Clases ANTES de la Aumentación ---")
for cls, count in class_counts.items():
    print(f"Clase '{cls}': {count} imágenes")

print(f"\nObjetivo: Balancear todas las clases a ~{target_count} imágenes.\n")

# Lista de funciones de aumentación que aplicaremos
augmentation_functions = [
    random_flip,
    # random_rotation,
    random_color_jitter,
    random_zoom
]

# --- BUCLE DE AUMENTACIÓN ---
for cls in classes:
    current_count = class_counts[cls]

    if current_count >= target_count:
        print(f"Clase '{cls}' ya está balanceada. Saltando.")
        continue

    images_to_generate = target_count - current_count
    class_path = os.path.join(input_folder, cls)
    image_files = os.listdir(class_path)

    print(f"Aumentando la clase '{cls}': generando {images_to_generate} imágenes nuevas...")

    for i in tqdm(range(images_to_generate), desc=f"Clase {cls}"):
        # 1. Elegir una imagen aleatoria de la clase para transformar
        random_image_name = random.choice(image_files)
        image_path = os.path.join(class_path, random_image_name)

        with Image.open(image_path).convert("RGB") as img:
            # 2. Aplicar una o más transformaciones aleatorias
            # Elegimos aplicar entre 1 y 3 transformaciones diferentes a cada nueva imagen
            num_transforms_to_apply = random.randint(1, 3)
            transforms_to_apply = random.sample(augmentation_functions, num_transforms_to_apply)

            augmented_image = img
            for func in transforms_to_apply:
                augmented_image = func(augmented_image)

            # 3. Crear un nuevo nombre de archivo y guardar la imagen
            base_name, ext = os.path.splitext(random_image_name)
            new_image_name = f"{base_name}_aug_{i}{ext}"
            save_path = os.path.join(class_path, new_image_name)

            augmented_image.save(save_path)

print("\n--- Aumentación Offline Completada ---")
final_counts = {cls: len(os.listdir(os.path.join(input_folder, cls))) for cls in classes}
print("--- Conteo de Clases DESPUÉS de la Aumentación ---")
for cls, count in final_counts.items():
    print(f"Clase '{cls}': {count} imágenes")

--- Conteo de Clases ANTES de la Aumentación ---
Clase 'Fresh_Leaf': 708 imágenes
Clase 'Leaf_Reddening': 165 imágenes
Clase 'Leaf_Spot_Bacterial_Blight': 1243 imágenes
Clase 'Yellowish_Leaf': 75 imágenes

Objetivo: Balancear todas las clases a ~1290 imágenes.

Aumentando la clase 'Fresh_Leaf': generando 582 imágenes nuevas...


Clase Fresh_Leaf: 100%|██████████| 582/582 [00:31<00:00, 18.48it/s]


Aumentando la clase 'Leaf_Reddening': generando 1125 imágenes nuevas...


Clase Leaf_Reddening: 100%|██████████| 1125/1125 [02:00<00:00,  9.30it/s]


Aumentando la clase 'Leaf_Spot_Bacterial_Blight': generando 47 imágenes nuevas...


Clase Leaf_Spot_Bacterial_Blight: 100%|██████████| 47/47 [00:05<00:00,  8.04it/s]


Aumentando la clase 'Yellowish_Leaf': generando 1215 imágenes nuevas...


Clase Yellowish_Leaf: 100%|██████████| 1215/1215 [00:55<00:00, 22.03it/s]


--- Aumentación Offline Completada ---
--- Conteo de Clases DESPUÉS de la Aumentación ---
Clase 'Fresh_Leaf': 1290 imágenes
Clase 'Leaf_Reddening': 1290 imágenes
Clase 'Leaf_Spot_Bacterial_Blight': 1290 imágenes
Clase 'Yellowish_Leaf': 1290 imágenes


In [18]:
# --- Configuración ---
folders = ["data/train", "data/valid", "data/test"]
target_size = (224, 224)

print("--- Iniciando pre-procesamiento (redimensionamiento) de imágenes ---")

# --- Bucle de Procesamiento ---
for folder in folders:
    if not os.path.exists(folder):
        print(f"Advertencia: La carpeta {folder} no existe. Saltando.")
        continue
        
    for cls in classes:
        class_folder = os.path.join(folder, cls)
        if not os.path.exists(class_folder):
            print(f"Advertencia: La carpeta de clase {class_folder} no existe. Saltando.")
            continue

        print(f"Procesando imágenes en: {class_folder}")
        for img_name in os.listdir(class_folder):
            img_path = os.path.join(class_folder, img_name)
            try:
                with Image.open(img_path) as img:
                    # Asegurarse de que la imagen esté en modo RGB
                    if img.mode != 'RGB':
                        img = img.convert('RGB')
                    
                    # Redimensionar la imagen
                    img_resized = img.resize(target_size, Image.Resampling.BICUBIC)
                    
                    # Guardar la imagen redimensionada, sobrescribiendo la original
                    img_resized.save(img_path)
            except Exception as e:
                print(f"Error procesando la imagen {img_path}: {e}")

print("--- Pre-procesamiento completado ---")


--- Iniciando pre-procesamiento (redimensionamiento) de imágenes ---
Procesando imágenes en: data/train\Fresh_Leaf
Procesando imágenes en: data/train\Leaf_Reddening
Procesando imágenes en: data/train\Leaf_Spot_Bacterial_Blight
Procesando imágenes en: data/train\Yellowish_Leaf
Procesando imágenes en: data/valid\Fresh_Leaf
Procesando imágenes en: data/valid\Leaf_Reddening
Procesando imágenes en: data/valid\Leaf_Spot_Bacterial_Blight
Procesando imágenes en: data/valid\Yellowish_Leaf
Procesando imágenes en: data/test\Fresh_Leaf
Procesando imágenes en: data/test\Leaf_Reddening
Procesando imágenes en: data/test\Leaf_Spot_Bacterial_Blight
Procesando imágenes en: data/test\Yellowish_Leaf
--- Pre-procesamiento completado ---
